In [1]:
!pip install git+https://github.com/huggingface/transformers.git -q -U # transformers version:  4.37.0
!pip install git+https://github.com/huggingface/accelerate.git -q -U # accelerate version:  0.27.0
!pip install bitsandbytes # bitsandbytes version:  0.42.0
!pip install git+https://github.com/huggingface/peft.git -q -U # peft version: 0.7.2
!pip install torch
!pip install --upgrade bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:0000:0100:01


In [2]:
from huggingface_hub import notebook_login as hfl
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

2024-05-26 06:46:19.248001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 06:46:19.248098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 06:46:19.371724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
hfl()

In [5]:
model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00017.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00002-of-00017.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00003-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00014-of-00017.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00015-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00016-of-00017.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
example = {
    "instruction": "Write a function that implements post-order traversal for an input binary tree as a dictionary or map in Python."
}

def formatPromptFromDictionary(example):
    prompt = f"<s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.{example['instruction']}[/INST]"
    
    return prompt

examplePrompt = formatPromptFromDictionary(example)
print(f'Prompt Template:\n{examplePrompt}')

Prompt Template:
<s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.Write a function that implements post-order traversal for an input binary tree as a dictionary or map in Python.[/INST]


In [9]:
from datetime import datetime
import re

def model_seq_gen(model, temp=0.7) : 
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
        
        start = datetime.now()
        
        sequences = pipe(
            examplePrompt,
            do_sample=True,
            max_new_tokens=100, 
            temperature=temp, 
            top_k=50, 
            top_p=0.95,
            num_return_sequences=1
        )
        
        output = sequences[0]['generated_text'].split("[/INST]")[1]
        stop = datetime.now()
        time_taken = stop-start
        
        print(f"Execution Time: {time_taken}s")
        
        return output

In [10]:
output = model_seq_gen(model, 0.5)
display(output)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Execution Time: 0:01:02.013244


"Here is the Python function that implements post-order traversal for a binary tree represented as a dictionary or map:\n\n```python\ndef post_order_traversal(tree):\n    result = []\n    if tree:\n        result.extend(post_order_traversal(tree.get('left')))\n        result.extend(post_order_traversal(tree.get('right')))\n        result.append(tree.get('value'))\n    return result\n```\n\nThis function takes a dictionary or map representing a binary tree as input. The tree is assumed to have a 'left"